# **🔥 Build a YouTube Comments Sentiment Analyzer with HuggingFace model**

## **✨ Powered by: MHS Bytebits 🤖**

# **📚 Importing Libraries**

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import plotly.express as px
import time

# 📝 **Scraping Comments of Video 🎥** 

In [ ]:
def get_youtube_comments(video_url, max_comments=10):
    # Setup the driver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run headless (no window)
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    print("Opening video...")
    driver.get(video_url)
    time.sleep(5)

    # Scroll to load comments
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height or len(driver.find_elements(By.XPATH, '//*[@id="content-text"]')) >= max_comments:
            break
        last_height = new_height

    print("Extracting comments...")
    comments_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
    comments = [elem.text for elem in comments_elements[:max_comments]]

    driver.quit()
    return comments

# Usage
video_url = input("Enter YouTube video URL: ")
comments = get_youtube_comments(video_url, max_comments=1000)
print(f"Extracted {len(comments)} comments:\n")
for i, c in enumerate(comments, 1):
    print(f"{i}. {c}\n")

# 🛠️ **Converting Comments into Pandas DataFrame** 

In [ ]:
df = pd.DataFrame({'Comment': comments})
df.head()

# **🤗 HuggingFace Model Integration 🤖** 

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
result = pipe(comments)
print(result)


# **📊 Plotting the Results 🎨** 

In [ ]:
df = pd.DataFrame({
    'Comment': comments,
    'Label': [res['label'] for res in result],
    'Score': [res['score'] for res in result]
})

# Save DataFrame to CSV (optional)
# df.to_csv('youtube_comments_sentiment.csv', index=False)

# Plot sentiment distribution as a pie chart using Plotly
sentiment_counts = df['Label'].value_counts()

fig = px.pie(
    names=sentiment_counts.index,
    values=sentiment_counts.values,
    color=sentiment_counts.index,
    color_discrete_map={'POSITIVE': 'lime', 'NEGATIVE': 'red'}, 
    title="YouTube Comments Sentiment Distribution"
)

fig.update_layout(
    template="plotly_dark",
    paper_bgcolor='black',
    plot_bgcolor='black',
    title_font=dict(size=20, color='white'),
    legend=dict(font=dict(color='white'))
)

fig.show()


# **Thank You! 🎉** 

Please check out the YouTube video by clicking the button below! ⬇️

[▶️ Watch Video](https://youtu.be/0m1Cg1RmDaU?si=k1ZIj1lYKhtge4jL)